In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,GradientBoostingClassifier,GradientBoostingRegressor,AdaBoostClassifier,AdaBoostRegressor
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from xgboost import XGBClassifier,XGBRFRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline

import tensorflow
from tensorflow import keras
from keras.layers import Dense,BatchNormalization,Dropout
from keras import Sequential

In [3]:
dataset = pd.read_csv(r"k-12.csv")

In [4]:
dataset['promoted'].value_counts()

promoted
No     5143
Yes    1300
Name: count, dtype: int64

In [5]:
dataset1 = dataset.drop(columns=['Name','Country','State','City','Course Name','Material Name','recommended_material','Gender'])

In [6]:
dataset1

,Age,Parental_Education_Level,Earning Class,Level,Course Level,Material Level,Previous_Scores,Assesment Score,IQ,Attendance,Study Time,promoted
0,10,High School,Low,Elementary,Medium,High,73,67,93.15,84,4.02,No
1,11,College,Medium,Middle School,Medium,Medium,59,61,81.45,64,3.66,No
2,5,Postgraduate,Medium,Kindergarten,Medium,Medium,91,74,107.55,98,4.44,Yes
3,15,High School,Medium,High School,Medium,Medium,98,71,108.00,89,4.26,Yes
4,10,College,Medium,Elementary,High,Medium,65,70,92.25,92,4.20,No
...,...,...,...,...,...,...,...,...,...,...,...,...
6438,18,High School,High,High School,Medium,Medium,76,68,95.40,69,4.08,No
6439,5,High School,Low,Kindergarten,High,Medium,81,69,98.55,76,4.14,Yes
6440,10,Postgraduate,Low,Elementary,Medium,Low,65,68,90.45,90,4.08,No
6441,5,High School,Low,Kindergarten,Medium,High,91,68,102.15,86,4.08,No


In [7]:
x = dataset1.drop(columns=['promoted'])
y = dataset1['promoted']

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
lb = LabelEncoder()

y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

lb = LabelEncoder()

transformer1 = ColumnTransformer(
    transformers=[
        ('tf1', OrdinalEncoder(categories=[
            ['High School', 'College', 'Postgraduate'],  # Categories for 'Parental Education Level'
            ['Low', 'Medium', 'High'],  # Categories for 'Earning Class'
            ['Kindergarten', 'Elementary', 'Middle School', 'High School'],  # Categories for 'Level'
            ['Low', 'Medium', 'High'],  # Categories for 'Course Level'
            ['Low', 'Medium', 'High']  # Categories for 'Material Level'
        ]), ['Parental_Education_Level', 'Earning Class', 'Level', 'Course Level', 'Material Level'])  # Ordinal columns
    ],
    remainder='passthrough'
)

pipeline = Pipeline(
    [('Preprocess',transformer1),
     ('Scaling',StandardScaler()),
     ('Classifier',RandomForestClassifier())
    ]
)

In [11]:
pipeline.fit(x_train,y_train)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('Preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('tf1',
                                                  OrdinalEncoder(categories=[['High '
                                                                              'School',
                                                                              'College',
                                                                              'Postgraduate'],
                                                                             ['Low',
                                                                              'Medium',
                                                                              'High'],
                                                                             ['Kindergarten',
                                                                              'Elementary',
                                                                              'Middle '
                                                                              'School',
                                                                              'High '
                                                                              'School'],
                                                                             ['Low',
                                                                              'Medium',
                                                                              'High'],
                                                                             ['Low',
                                                                              'Medium',
                                                                              'High']]),
                                                  ['Parental_Education_Level',
                                                   'Earning Class', 'Level',
                                                   'Course Level',
                                                   'Material Level'])])),
                ('Scaling', StandardScaler()),
                ('Classifier', RandomForestClassifier())])

In [12]:
y_pred = pipeline.predict(x_test)

In [13]:
accuracy_score(y_test,y_pred)

1.0

In [22]:
x_train['Level'].value_counts()

Level
Kindergarten     1320
High School      1308
Elementary       1264
Middle School    1262
Name: count, dtype: int64

In [15]:
# Predicting Whether student will be Promoted or Not
input_data = pd.DataFrame([[
    6,  'Postgraduate', 'Low', 'Kindergarten', 
    'Medium', 'Medium', 89, 69, 132.15, 86, 5.10
]], columns=x_train.columns)

# Make prediction
prediction = pipeline.predict(input_data)

if prediction == 1:
    print("Student will be Promoted")
else:
    print("Student Not Promoted")    

Student will be Promoted


In [ ]:
# import pickle
# pickle.dump(pipeline,open('promotion.pkl','wb'))